<a href="https://colab.research.google.com/github/emmarogge/cs280r/blob/master/student_classification_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Text classification with Colab and PyTorch

Emma Rogge, Tasha Schoenstein & Zilin Ma

## Set up

###Import relevant libraries and dependencies

In [0]:
import torch
import torch.nn as nn
from torch import optim
from torchtext import data
import math
from matplotlib import pyplot as plt
import os
from collections import Counter

## GPU check, make sure to set runtime type to "GPU"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

### Read in data files from GitHub

In [0]:
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/train.nl
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/train.sql
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/test.nl
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/test.sql

##Data format

We're going to use `torchtext` to handle processing the data. This library is useful for processing and batching text data in Python. More information on `torchtext` can be found [in this tutorial](https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/).

To begin, we set up two instances of the PyTorch class  one for the natural-language queries and one for the SQL query intent labels.
Next, we create instances of the PyTorch [`data.Field`](https://torchtext.readthedocs.io/en/latest/data.html#fields) class for the input (text) and output (labels) fields.
This class contains common text-processing datatypes that can be converted to tensors.

In [0]:
# We set `batch_first` = True to ensure the data is batched before it is processed.
TEXT = data.Field(lower=True, sequential=True, include_lengths=False, batch_first=True, tokenize="spacy") 
LABEL = data.Field(batch_first=True, sequential=False, unk_token=None)

###Implement torchtext Dataset
Implement the class below to prepare the data for classification. It is highly recommended that you make use of the [`Example class`](https://github.com/pytorch/text/blob/master/torchtext/data/example.py) to store each corresponding text and label.

#### Hints:
- Start by populating a list with each processed, tokenized query in your dataset.
- Each text field object in your list should then have its `label` field populated with the appropriate label.
- Leverage the `__init__` method of the parent class, [`pytorch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset).
- Utilize the PyTorch's `preprocess` method on your text field as you iterate--this method tokenizes the content of the 'text' field in an Example using `spacy`.

---

In [0]:
    ## Convert to standard format
class ATIS(data.Dataset):
    dirname = 'data'
    name = 'atis'

    def __init__(self, path, text_field, label_field, **kwargs):
        """Create an ATIS dataset instance given a path and fields.
        Arguments:
            path: Path to the data file
            text_field: The field that will be used for text data.
            label_field: The field that will be used for label data.
            Remaining keyword arguments: Passed to the constructor of
                data.Dataset.
        """
        super(ATIS, self).__init__(examples, fields, **kwargs)
        #TODO: Complete the implementation of this method.


    # Simple function to get question labels from query
    def _get_label_from_query(self, query):
        #TODO: Implement this method.

    @classmethod
    def splits(cls, text_field, label_field, path='./',
               train='train', validation='dev', test='test',
               **kwargs):
        """Create dataset objects for splits of the ATIS dataset.
        Arguments:
            text_field: The field that will be used for the sentence.
            label_field: The field that will be used for label data.
            root: The root directory that the dataset's zip archive will be
                expanded into; therefore the directory in whose trees
                subdirectory the data files will be stored.
            train: The filename of the train data. Default: 'train.txt'.
            validation: The filename of the validation data, or None to not
                load the validation set. Default: 'dev.txt'.
            test: The filename of the test data, or None to not load the test
                set. Default: 'test.txt'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.
        """
        train_data = None if train is None else cls(
            os.path.join(path, train), text_field, label_field, **kwargs)
        val_data = None if validation is None else cls(
            os.path.join(path, validation), text_field, label_field, **kwargs)
        test_data = None if test is None else cls(
            os.path.join(path, test), text_field, label_field, **kwargs)
        return tuple(d for d in (train_data, val_data, test_data)
                     if d is not None)

###Implement tortchtext Iterators





We will use the `ATIS.splits` class method to build the `ATIS` instances for train and test data. This method splits the data into either two (train & test) or three (train, test, validation) subsets.

In [0]:
# Make splits for data
train_data, test_data = ATIS.splits(TEXT, LABEL, validation=None)

# Build vocabulary for data fields
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

Once the data is processed we build the vocabulary and then construct iterators which loop over the datasets in batches. This will be important for SGD for logistic regression and for other models later in the course.

In [0]:
# Make iterator for splits
BATCH_SIZE = 32
train_iter = data.BucketIterator(
    train_data,
    batch_size=BATCH_SIZE,
    device=device)

test_iter = data.Iterator(test_data, batch_size=BATCH_SIZE, sort=False, device=device)

### Bag-of-Words Text Representation
####Your Naive Bayes, logistic regression and MLP classifiers will use a bag of words representation for the data. The `torchtext` iterators output tokenized natural language, which you must convert into a bag of words representation. 
---
####HINT: Your vocabulary should be derived ONLY from your training set, not your entire dataset. Make certain that your bag-of-words representations account for this. You may have unknown words in your test set and we leave it up to you to decide the best way of handling this.

Additionally, you may find the following methods for tensor manipulation useful to ensure your tensors are of the appropriate dimensions.



* https://pytorch.org/docs/stable/torch.html#torch.unsqueeze
* https://pytorch.org/docs/stable/tensors.html#torch.Tensor.scatter_
* https://pytorch.org/docs/stable/torch.html#torch.sum



In [0]:
# Compute size of vocabulary
vocab_size = len(TEXT.vocab.itos)
labels_size = len(LABEL.vocab.itos)
print("Size of vocab: {}".format(vocab_size))

# Given a batch, provide a bag-of-words vector.
def batch_to_bow(batch, vocab_size):
    # TODO: Implement this method.
    return None

## Establish a majority baseline

By defining a lower bound on performance, we know at minimum what to expect from any reasonable system. A simple baseline for classification tasks is to measure the accuracy of prediction when the most common class is always predicted. 

**Write code in the cell below that, given train and test data, prints information concerning the majority baseline.**

HINT: Use the [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) class from Python's `collections` library.

In [0]:
def majority_baseline_accuracy(train, test):
    #TODO: Implement this method as described.
    return None

# Call the method to establish a baseline
majority_baseline_accuracy(train_data, test_data)

# Naive Bayes

Naive Bayes classification is based on the "naive" assumption that all features are independent. This dramatically reduces the number of parameters required for Bayesian classification, which utilizes Bayes' Theorem which utilizes known information ($P(Y)$, $P(X)$ and $(P(X_i|Y)$) to obtain the desired unknown probability of $P(Y|X)$. This is evaluated for each possible label and the label with greatest likelihood is the prediction for a given text. 

---
Let $ c_{NB} $ be the maximum value in a vector containing the conditional probabilities of label $c$ given each word in the text. Then, we can compute $c_{NB}$ by evaluating the probability of the label overall and the probability of the label given the presence of each word contained in a given text, as 
$$ c_{NB} = \text{argmax}_{c \in C} \left( \log P(c) + \sum_{w \in W}\log P(w|c) \right) $$

Where $c_{NB}$ is the naive Bayes classification of a bag of words, $C$ is the set of classifications, and $W$ is the bag of words.

We can calculate $P(c) = \frac{N_c}{N}$ where $N$ is the total number of data points in our training data and $N_c$ is the total number of data points in our training data with classification $c$. 

We can calculate $P(w_0 | c)$ using Laplace smoothing such that $$P(w_0 | c) = \frac{count(w_0, c) + 1}{\left( \sum_{w \in V} count(w,c)\right) + |V|}$$ where $V$ is the vocabulary.

----
##Below, implement the NaiveBayes class methods.
 

###1.  `train`: Populates the log probabilities table to contain $log(P(c))$ and $log(𝑃(𝑤_i|𝑐)) $ for each label for each word in the vocabulary.
###2.   `evaluate_performance`: Evaluates the performance of the model on given datset and prints accuracy.

In [0]:
class NaiveBayes():
    def __init__ (self, text, label):
        # TODO: Implement this method.
        
    def train(self, dataset):
        """
        Populates log probabilities table for training data.
        """
        #TODO: Implement this method.
    
    def evaluate_performance(self, dataset):
        """
        Takes a dataset and returns the model's performance that dataset.
        """
        #TODO: Implement this method.

## Putting it all together

If you have implemented the NaiveBayes class and associated methods, the following code will train the model on the training set and evaluates its performance on both train and test sets.

In [0]:
# Instantiate and train classifier
nb_classifier = NaiveBayes(TEXT, LABEL)
nb_classifier.train(train_data)

# Evaluate model performance
print("Train: ")
nb_classifier.evaluate_performance(train_data)
print("Test: ")
nb_classifier.evaluate_performance(test_data)

# Logistic Regression

Unlike Naive Bayes, logistic regression calculates the conditional probabilities directly. If we let $c\in C$ be a label, $\mathbf{w} \in W$ be a bag-of-words representation of a natural language query, $\mathbf{d}$ be weights in the model tied to the compatability of $c$ and $\mathbf{w}$, and $f$ is $\mathbf{d}^T \mathbf{w}$, we use the softmax to get: 
$$ p(c|\mathbf{w}, \mathbf{d})= \frac{\exp (f(\mathbf{w},c,\mathbf{d}))}{\sum_{c'\in C}\exp(f(\mathbf{w},c',\mathbf{d}))}. $$

The weights are learned in the process of training by using a loss function--here the cross entropy loss--to compare the results produced by the current version of the model and the target results. 
---
###Below, implement the LogisticRegression class methods.
####1.  `__init__` : Takes the TEXT and LABEL `data.Field` instances and initializes the model.
####2.  `forward` : Given an input tensor, performs the forward step of the logistic regression.
HINT: Your LogisticRegression implementation may inherit from PyTorch's [nn.Module](https://pytorch.org/docs/stable/nn.html#module) class.


In [0]:
class LogisticRegression(nn.Module):
    def __init__ (self, label, text):
        #TODO: Implement this method.

    def forward (self, input):
        #TODO: Implement this method.

### Implement the method `train` for the LogisticRegression model.
<b>Parameters:</b> LogisticRegression model, data iterator, criterion, optimizer and # of epochs.

Trains the model for n epochs with provided optimizer and learning rate.


In [0]:
# TODO: Implement the method.
def train (model, data_iter, criterion, optim, n_epochs = 8):
    loss_values = []
    epochs = []
    for epoch in range (n_epochs):
        # TODO: Complete the implementation such that the sanity check, 
        # an 'Epoch vs Loss' plot, is rendered when you run the notebook.
        epoch_loss = running_loss / len(train_data)
        loss_values.append(epoch_loss)
        epochs.append(epoch)

    p0 = plt.figure(0)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.plot(epochs, loss_values)
    plt.title("Epoch vs Loss")
    p0.show()

### Implement the method `evaluate_performance`.
This method takes a model & dataset, and returns the accuracy of the model on the dataset.

In [0]:
def evaluate_performance(model, data_iter):
    #TODO: Implement method.

## Putting it all together

If you have implemented the LogisticRegression class and associated methods, the following code will  train the model on the training set and evaluates its performance on both train and test sets.

In [0]:
# Instantiate classifier
logreg_model = LogisticRegression(LABEL, TEXT).to(device) 
print(logreg_model)

# Build criterion (loss), optimizer
loss = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(logreg_model.parameters(), lr=learning_rate)

# Train classifier model on training split
epochs = []
train_acc = []
test_acc = []
for n in range (5, 15):
    # Train model for n epochs
    train(logreg_model, train_iter, loss, optimizer, n)
    epochs.append(n)
    # Evaluate model performance on training, test splits
    train_acc.append(evaluate_performance(logreg_model, test_iter))
    test_acc.append(evaluate_performance(logreg_model, test_iter))

# Graph loss vs accuracy for train, test
p1 = plt.figure(1)
plt.title("Epochs vs Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
train_line = plt.plot(epochs, train_acc, 'b', label="Train")
test_line = plt.plot(epochs, test_acc, 'r', label="Test")
plt.legend()
p1.show()

# Multilayer Perceptron

An MLP is composed of at least three fully connected layers of nodes, referred to as the input, output and "hidden" layers. Learning occurs by adjusting the connection weights between nodes based on the amount of error in the output compared to the prediction. 

---
Let the degree of error in an output node $j$ in the $n$th training query be $e_j(n) = d_j(n) - y_j(n)$, where $d$ is the true label and $y$ is the predicted label.

Then we can adjust the weights to minimize the entire output layer's cumulative error:
$$\mathcal{E}(n)=\frac{1}{2}\sum_j e_j^2(n)$$

The change in each weight according to gradient descent is 
$$\Delta w_{ji} (n) = -\eta\frac{\partial\mathcal{E}(n)}{\partial v_j(n)} y_i(n)$$.

---
##Implement the methods of the class MultiLayerPerceptron below.
####1.  `__init__` : Takes the TEXT and LABEL `data.Field` instances and initializes the model.
####2.  `forward` : Given an input tensor, performs the forward steps of the multilayer perceptron.

In [0]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, label, text, n_hidden=128):
        #TODO: Implement this method.

    def forward(self, input):
        #TODO: Implement this method.

### Implement the method `train` for the MultiLayerPerceptron model.
<b>Parameters:</b> MultiLayerPerceptron model, data iterator, criterion, optimizer and # of epochs.

Trains the model for n epochs with provided optimizer and learning rate.


In [0]:
def train(model, data_iter, criterion, optim, n_epochs = 10):
    loss_values = []
    epochs = []
    for epoch in range (n_epochs):
        curr_loss = 0.0
        running_loss = 0.0
        c_num = 0
        total = 0
        for index, batch in enumerate(data_iter):
            # TODO: Complete the implementation such that the sanity check, 
            # an 'Epoch vs Loss' plot, is rendered when you run the notebook.
            epoch_loss = running_loss / len(train_data)
            loss_values.append(epoch_loss)
            epochs.append(epoch)
    p2 = plt.figure(2)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.plot(epochs, loss_values)
    plt.title("Epoch vs Loss")
    p2.show()

### Implement the method `evalaute_performance`.
This method takes a model & dataset, and returns the accuracy of the model on the dataset.

In [0]:
def evaluate_performance(model, data_iter):
    c_num = 0
    total = 0
    with torch.no_grad():
        for index, batch in enumerate(data_iter):
            # Input and target
            input = batch_to_bow(batch.text, model.get_vocab_size())
            target = batch.label.long()

            # Feed the input and hidden state to the model
            scores = model(input)
            
            # Determine the index of the maximum value for each test item
            predictions = torch.argmax(scores, dim=1)

            # Prepare to compute the accuracy
            total += len(target)
            c_num += (predictions == target).sum().item()

            # Return the accuracy
            print("Accuracy: {}".format(float(c_num)/total))
            return float (c_num)/total

## Putting it all together

If you have implemented the MultiLayerPerceptron class and associated methods, the following code will  train the model on the training set and evaluates its performance on both train and test sets.


In [0]:
# Instantiate classifier
mlp_model = MultiLayerPerceptron(LABEL, TEXT).to(device)
print(mlp_model)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  
learning_rate = 0.01
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=learning_rate)

# Train classifier model on training split
epochs = []
train_acc = []
test_acc = []
for n in range (5, 50, 5):
    # Train model for n epochs
    train(mlp_model, train_iter, loss, optimizer, n)
    epochs.append(n)
    # Evaluate model performance on training, test splits
    train_acc.append(evaluate_performance(mlp_model, train_iter))
    test_acc.append(evaluate_performance(mlp_model, test_iter))

# Graph loss vs accuracy for train, test
p3 = plt.figure(3)
plt.title("MLP - Epochs vs Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
train_line = plt.plot(epochs, train_acc, 'b', label="Train")
test_line = plt.plot(epochs, test_acc, 'r', label="Test")
plt.legend()
p3.show()